In [45]:
import nwslpy
import pandas as pd

In [20]:
n = 10

In [3]:
df = nwslpy.load_matches()
df

,home_team_id,away_team_id,kickoff,time,stadium,city,year,season,home_team_score,away_team_score,home_penalty_score,away_penalty_score,won_on_penalties,last_updated
match_id,,,,,,,,,,,,,,
washington-spirit-vs-boston-breakers-2016-04-16,9,1,2016-04-16 19:00:00,19:00:00,NaN,NaN,2016,2016,1,0,0,0,False,2023-03-12
houston-dash-vs-chicago-red-stars-2016-04-16,4,2,2016-04-16 20:30:00,20:30:00,Shell Energy Stadium,"Houston, Texas",2016,2016,3,1,0,0,False,2023-03-12
fc-kansas-city-vs-western-new-york-flash-2016-04-16,3,10,2016-04-16 20:30:00,20:30:00,Children's Mercy Park,"Kansas City, Kansas",2016,2016,0,1,0,0,False,2023-03-12
seatle-reign-fc-vs-sky-blue-fc-2016-04-17,6,5,2016-04-17 19:00:00,19:00:00,UW Medicine Pitch at Memorial Stadium,Seattle,2016,2016,1,2,0,0,False,2023-03-12
portland-thorns-fc-vs-orlando-pride-2016-04-17,8,7,2016-04-17 22:00:00,22:00:00,Providence Park,"Portland, Oregon",2016,2016,2,1,0,0,False,2023-03-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nj-ny-gotham-fc-vs-north-carolina-courage-2023-08-05,5,11,2023-08-05 19:30:00,19:30:00,Red Bull Arena,"Harrison, New Jersey",2023,2023 Challenge Cup,0,0,0,0,False,2023-05-28
chicago-red-stars-vs-houston-dash-2023-08-05,2,4,2023-08-05 20:00:00,20:00:00,SeatGeek Stadium,"Bridgeview, Illinois",2023,2023 Challenge Cup,0,0,0,0,False,2023-05-28
ol-reign-vs-portland-thorns-fc-2023-08-06,6,8,2023-08-06 18:00:00,18:00:00,Lumen Field,"Seattle, Washington",2023,2023 Challenge Cup,0,0,0,0,False,2023-05-28


In [28]:
df_teams = nwslpy.load_teams()

In [39]:
seasons = ["2022", "2023"]
df = df[df["season"].isin(seasons)]
teams = df["home_team_id"].unique()
teams.sort()

In [40]:
def get_result(row, team):
    if team == row["home_team_id"]:
        if row["home_team_score"] > row["away_team_score"]:
            return 3
        elif row["home_team_score"] < row["away_team_score"]:
            return 0
        else:
            return 1
    else:
        if row["home_team_score"] > row["away_team_score"]:
            return 0
        elif row["home_team_score"] < row["away_team_score"]:
            return 3
        else:
            return 1


def get_goals(row, team):
    if team == row["home_team_id"]:
        return row["home_team_score"]
    else:
        return row["away_team_score"]


def get_opp_goals(row, team):
    if team == row["home_team_id"]:
        return row["away_team_score"]
    else:
        return row["home_team_score"]


def load_data(season, team):
    df_subset = df[
        (df["season"] == season)
        & ((df["home_team_id"] == team) | (df["away_team_id"] == team))
    ]
    df_subset = df_subset.sort_values("kickoff")
    df_subset = df_subset.iloc[0:n]
    df_subset["points"] = df_subset.apply(lambda r: get_result(r, team), axis=1)
    df_subset["goals"] = df_subset.apply(lambda r: get_goals(r, team), axis=1)
    df_subset["opp_goals"] = df_subset.apply(lambda r: get_opp_goals(r, team), axis=1)
    name = df_teams.loc[team]
    agg = df_subset.sum()
    return [
        season,
        name["team_abbreviation"],
        agg["points"],
        agg["goals"],
        agg["opp_goals"],
        agg["goals"] - agg["opp_goals"],
    ]


load_data("2023", 2)

<ipython-input-40-72b266513ae9>:43: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  agg = df_subset.sum()


['2023', 'CHI', 8, 12, 22, -10]

In [46]:
data = []

for season in seasons:
    for team in teams:
        data.append(load_data(season, team))

<ipython-input-40-72b266513ae9>:43: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  agg = df_subset.sum()


In [50]:
out = pd.DataFrame(
    data,
    columns=[
        "season",
        "team",
        "points",
        "goals_for",
        "goals_against",
        "goal_differential",
    ],
)
out.to_csv("data.csv", index=False)
out

,season,team,points,goals_for,goals_against,goal_differential
0,2022,CHI,19,17,10,7
1,2022,HOU,15,16,12,4
2,2022,NJY,12,7,18,-11
3,2022,RGN,16,9,6,3
4,2022,ORL,9,11,26,-15
5,2022,POR,17,22,8,14
6,2022,WAS,9,10,12,-2
7,2022,NC,9,17,19,-2
8,2022,KCC,12,10,14,-4
9,2022,LOU,10,12,16,-4
